# In silico perturbation experiment

### 事前準備

以下のファイルの変数にパスを指定

tokenizer.pyのTOKEN_DICTIONARY_FILE

in_silico_perturber_statsのGENE_NAME_ID_DICTIONARY_FILE

### 必要なモジュールのインポート

In [ ]:
# ==== Core ====
import sys, importlib.util, types
from pathlib import Path
from importlib import import_module   # ★NEW
import os

# ==== 1) ユーザ設定 ====
REPO_ROOT = "path/to/geneformer/"  # geneformer/ の親
PATH_TOKEN_DICT   = Path("/path/to/MLM-re_token_dictionary_v1.pkl")
PATH_GENE_NAME2ID = Path("/path/to/MLM-re_token_dictionary_v1_GeneSymbol_to_EnsemblID.pkl")

# ==== 2) 存在チェック ====
assert Path(REPO_ROOT, "geneformer").is_dir(), f"❌ geneformer not found under {REPO_ROOT}"
assert PATH_TOKEN_DICT.is_file(),   f"❌ Token dict not found: {PATH_TOKEN_DICT}"
assert PATH_GENE_NAME2ID.is_file(), f"❌ GeneName->Ensembl dict not found: {PATH_GENE_NAME2ID}"

# ==== 3) 既存の geneformer を掃除し、sys.path を通す ====
for k in list(sys.modules.keys()):
    if k == "geneformer" or k.startswith("geneformer."):
        del sys.modules[k]
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

# ==== 4) ダミー geneformer パッケージを作成（相対importに必要）====
if "geneformer" not in sys.modules:
    geneformer_pkg = types.ModuleType("geneformer")
    # 位置情報を埋めて unknown location を回避（★NEW）
    geneformer_pkg.__file__ = str(Path(REPO_ROOT, "geneformer", "__init__.py"))
    geneformer_pkg.__path__ = [str(Path(REPO_ROOT, "geneformer").resolve())]
    sys.modules["geneformer"] = geneformer_pkg

def load_as(name: str, file_path: str, predefine: dict = None):
    """file_path を 'name' という完全修飾名でロード。exec前に predefine をglobalsへ注入。"""
    spec = importlib.util.spec_from_file_location(name, file_path)
    mod = importlib.util.module_from_spec(spec)
    if predefine:
        for k, v in predefine.items():
            setattr(mod, k, v)
    spec.loader.exec_module(mod)  # type: ignore
    sys.modules[name] = mod
    return mod

# ==== 5) サブモジュールのパス ====
PATH_TOKENIZER_PY = str(Path(REPO_ROOT, "geneformer", "tokenizer.py"))
PATH_ISP_PY       = str(Path(REPO_ROOT, "geneformer", "in_silico_perturber.py"))
PATH_ISP_STATS_PY = str(Path(REPO_ROOT, "geneformer", "in_silico_perturber_stats.py"))

# ==== 6) tokenizer を先にロードして辞書パスを設定 ====
tokenizer_mod = load_as("geneformer.tokenizer", PATH_TOKENIZER_PY,
                        predefine={"TOKEN_DICTIONARY_FILE": str(PATH_TOKEN_DICT)})
# 念のためエイリアス名にも反映
for alias in ("token_dictionary_file", "TOKEN_DICT_PATH"):
    if hasattr(tokenizer_mod, alias):
        setattr(tokenizer_mod, alias, str(PATH_TOKEN_DICT))

print("✅ tokenizer.TOKEN_DICTIONARY_FILE =", tokenizer_mod.TOKEN_DICTIONARY_FILE)

# ==== 7) 依存順に in_silico_perturber → stats をロード ====
isp_mod = load_as("geneformer.in_silico_perturber", PATH_ISP_PY)
# 既定引数で参照される定数を事前定義してから exec させる
ispstats_mod = load_as("geneformer.in_silico_perturber_stats", PATH_ISP_STATS_PY,
                       predefine={
                           "GENE_NAME_ID_DICTIONARY_FILE": str(PATH_GENE_NAME2ID)
                       })

# ==== 7.5) 親パッケージへ “再エクスポート” を生やす（★NEW）====
pkg = sys.modules["geneformer"]
# サブモジュール参照
setattr(pkg, "tokenizer", tokenizer_mod)
setattr(pkg, "in_silico_perturber", isp_mod)
setattr(pkg, "in_silico_perturber_stats", ispstats_mod)

# クラス/関数を親直下に再エクスポート
setattr(pkg, "InSilicoPerturber", isp_mod.InSilicoPerturber)
# ISP_device も親から使いたいなら
setattr(pkg, "ISP_device", getattr(isp_mod, "ISP_device", None))
setattr(pkg, "InSilicoPerturberStats", ispstats_mod.InSilicoPerturberStats)

# collator 由来クラスも再エクスポート
collator_mod = import_module("geneformer.collator_for_classification")
setattr(pkg, "DataCollatorForCellClassification",
        getattr(collator_mod, "DataCollatorForCellClassification"))

# （以降、通常どおり import 可能）

# ==== 8) 以降は通常どおり import して利用 ====
from geneformer import InSilicoPerturberStats, InSilicoPerturber, DataCollatorForCellClassification
from geneformer import ISP_device  # 親から引けるようになった

# ==== 9) Stats インスタンスをNotebook設定で固定 ====
isp_stats = InSilicoPerturberStats(
    mode="mixture_model",
    combos=0,
    anchor_gene=None,
    cell_states_to_model=None,
    token_dictionary_file=str(PATH_TOKEN_DICT),
    gene_name_id_dictionary_file=str(PATH_GENE_NAME2ID),
)

print("✅ InSilicoPerturberStats ready (paths injected)")


✅ tokenizer.TOKEN_DICTIONARY_FILE = /home/pj25000162/ku50001814/pkl_data/MLM-re_token_dictionary_v1.pkl
✅ InSilicoPerturberStats ready (paths injected)


### 処理中のプログレスバーの表示設定

In [4]:
os.environ["TQDM_DISABLE_IPYWIDGETS"] = "1"

### ファイルのパスなどの設定

In [ ]:
import os
RESULT_DIR_NAME = "/path/to/result_dir"
# If you choice "CellClassifier", you choice fine tuning model. If you choice "Pretrained", you choice pretrained model.

PRE_TRAIN_MODEL_DIR = "/path/to/mouse-geneformer_pretrain_model"
DATASET_NAME = "/path/to/dataset"

#利用するGPUの番号を指定（複数の場合はカンマ区切り）
GPU_NUMBER = [0]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(s) for s in GPU_NUMBER])
os.environ["NCCL_DEBUG"] = "INFO"

## In silicoを行うためにモデルをファインチューニング

In [6]:
from datasets import load_from_disk
import pickle
import numpy as np
import glob
import tqdm
from tqdm import tqdm_notebook as tqdm

# load dataset
train_dataset = load_from_disk(DATASET_NAME)

print(np.unique(train_dataset["disease"]))
print(train_dataset)
organ_data = train_dataset["organ_major"][0]
rmfiles = glob.glob(DATASET_NAME+"/cache*")

if rmfiles == [] :
    print("not exist cache files")
else :
    for tqdm_i2, rmfile in zip(tqdm(rmfiles, desc='remove files loop'), rmfiles) :
        os.remove(rmfile)
    print("Finished removeing cache file in it !!")

['FD' 'FO' 'FOD' 'FS' 'MC' 'MCD' 'MS']
Dataset({
    features: ['cell_types', 'disease', 'input_ids', 'length', 'organ_major'],
    num_rows: 18373
})


/tmp/4119392_U1RESU4K/ipykernel_69990/1331688348.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for tqdm_i2, rmfile in zip(tqdm(rmfiles, desc='remove files loop'), rmfiles) :


remove files loop:   0%|          | 0/86 [00:00<?, ?it/s]

Finished removeing cache file in it !!


In [7]:
# disease classification
from collections import Counter

dataset_list = []
evalset_list = []
organ_list = []
target_dict_list = []

for organ in Counter(train_dataset["organ_major"]).keys():
    # collect list of tissues for fine-tuning (immune and bone marrow are included together)
    
    organ_ids = [organ]
    organ_list += [organ]
    
    print(organ)

    # filter datasets for given organ
    def if_organ(example):
        return example["organ_major"] in organ_ids
    trainset_organ = train_dataset.filter(if_organ, num_proc=16)
    
    # per scDeepsort published method, drop cell types representing <0.5% of cells
    celltype_counter = Counter(trainset_organ["disease"])
    total_cells = sum(celltype_counter.values())
    cells_to_keep = [k for k,v in celltype_counter.items() if v>(0.005*total_cells)]
    def if_not_rare_celltype(example):
        return example["disease"] in cells_to_keep
    trainset_organ_subset = trainset_organ.filter(if_not_rare_celltype, num_proc=16)
      
    # shuffle datasets and rename columns
    trainset_organ_shuffled = trainset_organ_subset.shuffle(seed=42)
    trainset_organ_shuffled = trainset_organ_shuffled.rename_column("disease","label")
    trainset_organ_shuffled = trainset_organ_shuffled.remove_columns("organ_major")
    
    # create dictionary of cell types : label ids
    target_names = list(Counter(trainset_organ_shuffled["label"]).keys())
    target_name_id_dict = dict(zip(target_names,[i for i in range(len(target_names))]))
    target_dict_list += [target_name_id_dict]
    
    # change labels to numerical ids
    def classes_to_ids(example):
        example["label"] = target_name_id_dict[example["label"]]
        return example
    labeled_trainset = trainset_organ_shuffled.map(classes_to_ids, num_proc=16)
    
    # create 80/20 train/eval splits
    labeled_train_split = labeled_trainset.select([i for i in range(0,round(len(labeled_trainset)*0.8))])
    labeled_eval_split = labeled_trainset.select([i for i in range(round(len(labeled_trainset)*0.8),len(labeled_trainset))])
    
    # filter dataset for cell types in corresponding training set
    trained_labels = list(Counter(labeled_train_split["label"]).keys())
    def if_trained_label(example):
        return example["label"] in trained_labels
    labeled_eval_split_subset = labeled_eval_split.filter(if_trained_label, num_proc=16)

    dataset_list += [labeled_train_split]
    evalset_list += [labeled_eval_split_subset]

nan


Filter (num_proc=16):   0%|          | 0/18373 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/18373 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/18373 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/3675 [00:00<?, ? examples/s]

In [8]:
trainset_dict = dict(zip(organ_list,dataset_list))
traintargetdict_dict = dict(zip(organ_list,target_dict_list))
evalset_dict = dict(zip(organ_list,evalset_list))

print(trainset_dict)
print(traintargetdict_dict)
print(evalset_dict)

{'nan': Dataset({
    features: ['cell_types', 'label', 'input_ids', 'length'],
    num_rows: 14698
})}
{'nan': {'MCD': 0, 'FO': 1, 'FOD': 2, 'FD': 3, 'MC': 4, 'FS': 5, 'MS': 6}}
{'nan': Dataset({
    features: ['cell_types', 'label', 'input_ids', 'length'],
    num_rows: 3675
})}


In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)
    pre = precision_score(labels, preds, average='macro')
    rec = recall_score(labels, preds, average='macro')
    macro_f1 = f1_score(labels, preds, average='macro')
    return {
      'accuracy': acc,
      'macro_precision': pre,
      'macro_recall': rec,
      'macro_f1': macro_f1
    }

In [10]:
# set model parameters
# max input size
max_input_size = 2**11  # 2048

# set training hyperparameters
# max learning rate
max_lr = 5e-5
# how many pretrained layers to freeze
freeze_layers = 0
# number gpus
num_gpus = 1
# number cpu cores
num_proc = 16
# batch size for training and eval
geneformer_batch_size = 12
# learning schedule
lr_schedule_fn = "linear" #"polynomial", "linear", "cosine"
# warmup steps
warmup_steps = 500
# number of epochs
epochs = 20
# optimizer
optimizer = "adamW"

In [11]:
# in silicoのみをしたい場合はスキップ

from transformers import BertConfig, BertForSequenceClassification
import datetime
import subprocess
from transformers.training_args import TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

for organ in organ_list:
    print(organ)
    organ_trainset = trainset_dict[organ]
    organ_evalset = evalset_dict[organ]
    organ_label_dict = traintargetdict_dict[organ]
    print(organ_label_dict)
    
    # set logging steps
    logging_steps = round(len(organ_trainset)/geneformer_batch_size/10)

    # reload pretrained model
    model = BertForSequenceClassification.from_pretrained(PRE_TRAIN_MODEL_DIR, 
                                                            num_labels=len(organ_label_dict.keys()),
                                                            output_attentions = False,
                                                            output_hidden_states = False).to("cuda")

    # define output directory path
    current_date = datetime.datetime.now()
    datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}"
    output_dir = os.path.join(RESULT_DIR_NAME, f"{datestamp}_mouse-geneformer_CellClassifier_{organ}_L{max_input_size}_B{geneformer_batch_size}_LR{max_lr}_LS{lr_schedule_fn}_WU{warmup_steps}_E{epochs}_O{optimizer}_F{freeze_layers}_ISP-{organ}/")

    saved_model_test = os.path.join(output_dir, f"pytorch_model.bin")
    if os.path.isfile(saved_model_test) == True:
        raise Exception("Model already saved to this directory.")
        
    # make output directory
    subprocess.call(f'mkdir {output_dir}', shell=True)

    # set training arguments
    training_args = {
        "learning_rate": max_lr,
        "do_train": True,
        "do_eval": True,
        "eval_strategy": "epoch",
        "save_strategy": "epoch",
        "logging_steps": logging_steps,
        "group_by_length": True,
        "length_column_name": "length",
        "disable_tqdm": False,
        "lr_scheduler_type": lr_schedule_fn,
        "warmup_steps": warmup_steps,
        "weight_decay": 0.001,
        "per_device_train_batch_size": geneformer_batch_size,
        "per_device_eval_batch_size": geneformer_batch_size,
        "num_train_epochs": epochs,
        "load_best_model_at_end": True,
        "output_dir": output_dir,
        #"max_position_embeddings": 2**11,
    }
    
    training_args_init = TrainingArguments(**training_args)

    # create the trainer
    trainer = Trainer(
        model=model,
        args=training_args_init,
        data_collator=DataCollatorForCellClassification(),
        train_dataset=organ_trainset,
        eval_dataset=organ_evalset,
        compute_metrics=compute_metrics
    )
    # train the cell type classifier
    trainer.train()
    predictions = trainer.predict(organ_evalset)
    with open(f"{output_dir}predictions.pickle", "wb") as fp:
        pickle.dump(predictions, fp)
    trainer.save_metrics("eval",predictions.metrics)
    trainer.save_model(output_dir)

nan
{'MCD': 0, 'FO': 1, 'FOD': 2, 'FD': 3, 'MC': 4, 'FS': 5, 'MS': 6}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/pj25000162/ku50001814/mouse-geneformer_pretrain_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
mkdir: ディレクトリ `/home/pj25000162/ku50001814/results/STMS_ENMC/251021_mouse-geneformer_CellClassifier_nan_L2048_B12_LR5e-05_LSlinear_WU500_E20_OadamW_F0_ISP-nan/' を作成できません: そのようなファイルやディレクトリはありません
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/pj25000162/ku50001814/src/Mouse-Geneformer/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clon

Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1
1,1.012800,0.935798,0.574422,0.475494,0.503183,0.430043
2,0.806800,0.711545,0.692245,0.679624,0.616004,0.583276
3,0.547000,0.544790,0.776871,0.734777,0.731764,0.698751
4,0.250100,0.353036,0.889252,0.890990,0.883738,0.883540
5,0.135500,0.263026,0.933878,0.933006,0.928871,0.928892
6,0.137300,0.227471,0.947483,0.945466,0.944345,0.944342
7,0.137500,0.214023,0.957007,0.956968,0.951679,0.953768
8,0.101600,0.196665,0.957551,0.956780,0.953403,0.954913
9,0.095100,0.211809,0.955102,0.947181,0.953376,0.949518
10,0.100300,0.184370,0.964082,0.963168,0.960146,0.961509


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home/pj25000162/ku50001814/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/pj25000162/ku50001814/src/Mouse-Geneformer/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home/pj25000162/ku50001814/src/Mouse-Geneformer/geneformer/collator_for_classification.py:581: UserWarning: To cop

## in silico　摂動実験

### データの読み込みと種類の確認

In [11]:
if not os.path.exists(RESULT_DIR_NAME):
    os.mkdir(RESULT_DIR_NAME)

# load disease dataset (xxx.dataset)
data = load_from_disk(DATASET_NAME)
print(data)
print("total cells: {}".format(len(data["length"])))

disease_types = np.unique(data["disease"])
print("types : ",disease_types)
print(disease_types.shape[0])


Dataset({
    features: ['cell_types', 'disease', 'input_ids', 'length', 'organ_major'],
    num_rows: 18373
})
total cells: 18373
types :  ['FD' 'FO' 'FOD' 'FS' 'MC' 'MCD' 'MS']
7


### あらかじめ，以下を設定する

select_perturb_type　：　in silicoの種類（削除の場合はdelete，活性化の場合はoverexpressを指定）

start_state : in silicoの初期状態

end_state : in silicoでの移動先

max_ncells : in silico実験を行う細胞数

In [12]:
# in silico perturbation in deletion mode to determine genes whose 
# deletion in the dilated cardiomyopathy (dcm) state significantly shifts
# the embedding towards non-failing (nf) state


# select_perturb_type: "delete","overexpress","inhibit","activate"

# "delete": delete gene from rank value encoding
# "overexpress": move gene to front of rank value encoding
# "inhibit": move gene to lower quartile of rank value encoding
# "activate": move gene to higher quartile of rank value encoding

FINETUNED_MODEL_DIR = "/home/pj25000162/ku50001814/results/251021_mouse-geneformer_CellClassifier_nan_L2048_B12_LR5e-05_LSlinear_WU500_E20_OadamW_F0_ISP-nan"
select_perturb_type="delete" # select perturb type in delete, overexpress, inhibit or activate, you wanna it.

start_state = 'MC'
end_state = 'FO'
alt_state = [] # If alt state is nothing, leave list empty.

# select model type in Pretrained, GeneClassifier or CellClassifier, you wanna it.
use_model_type = "CellClassifier"

# If you wanna perturb genes in your dataset, append its Ensembl ID in the list.
genes_to_perturb_list = [] # 

isp = InSilicoPerturber(perturb_type=select_perturb_type,
                        perturb_rank_shift=None,
                        genes_to_perturb="all" if len(genes_to_perturb_list) == 0 else genes_to_perturb_list,
                        combos=0,
                        anchor_gene=None,
                        model_type=use_model_type,
                        num_classes=disease_types.shape[0],
                        emb_mode="cell",
                        cell_emb_style="mean_pool",
                        filter_data=None,
                        cell_states_to_model={'state_key': 'disease', 
                                              'start_state':start_state, 
                                              'goal_state':end_state, 
                                              'alt_states': alt_state}, 
                        max_ncells=1000,
                        emb_layer=0,
                        forward_batch_size=100,
                        nproc=6)



### 実験開始

In [13]:
# outputs intermediate files from in silico perturbation
# in silicoが途中で止まった場合はここもスキップ
import pickle
import time 

start_time = time.time()

start_state = start_state.replace(" ","-")
end_state = end_state.replace(" ","-")


organ_data = data['organ_major'][0]
isp.perturb_data(FINETUNED_MODEL_DIR,
                 DATASET_NAME, # input data
                 RESULT_DIR_NAME,
                 "output_in-silico_SE{}_OR{}_ST{}_EN{}".format(select_perturb_type, organ_data, start_state, end_state)) # output prefix

end_time = time.time()
in_silico_time = int(end_time - start_time)
hours = in_silico_time // 3600
minutes = (in_silico_time % 3600) // 60
seconds = in_silico_time % 60

print(hours, "時間", minutes, "分", seconds, "秒")

Filter (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=6):   0%|          | 0/18373 [00:00<?, ? examples/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Map (num_proc=6):   0%|          | 0/2048 [00:00<?, ? examples/s]

...

Map (num_proc=6):   0%|          | 0/711 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/711 [00:00<?, ? examples/s]

Map:   0%|          | 0/364 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/364 [00:00<?, ? examples/s]

1 時間 24 分 29 秒


In [14]:


ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed="all" if len(genes_to_perturb_list) == 0 else genes_to_perturb_list,
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model={"state_key": "disease",
                                                        "start_state": start_state,
                                                        "goal_state": end_state,
                                                        "alt_states": alt_state},
                                 )


### 実験結果の集計

In [15]:
# extracts data from intermediate files and processes stats to output in final .csv

start_state = start_state.replace(" ","-")
end_state = end_state.replace(" ","-")

ispstats.get_stats(RESULT_DIR_NAME, # path to input data
                   None,
                   RESULT_DIR_NAME, 
                   "output_in-silico_SE{}_OR{}_ST{}_EN{}".format(select_perturb_type, organ_data, start_state, end_state)) # output prefix 


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/12478 [00:00<?, ?it/s]

  0%|          | 0/12478 [00:00<?, ?it/s]

/home/pj25000162/ku50001814/src/Mouse-Geneformer/geneformer/in_silico_perturber_stats.py:233: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cos_sims_full_df = pd.concat([cos_sims_full_df,cos_sims_df_i])
